In [107]:
import pandas as pd
import re
import json
from view import fetch_movie_info

In [2]:
movies = pd.read_csv('csv/movies.csv')
user_rating = pd.read_csv('csv/ratings.csv')


movies_genres = pd.DataFrame(columns = ['movieId', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
                                       'Drama', 'Fantasy', 'Film-noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                                       'Thriller', 'War', 'Western'])

genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
                                       'Drama', 'Fantasy', 'Film-noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                                       'Thriller', 'War', 'Western']

#movies_genres
index = 0
for i in movies.itertuples(index = False):
        has_genre = []
        has_genre.append(i[0])
        genre_movie = i[2].split('|')
        index_aux = 0
        for j in range(len(genres)):
            has = False
            for k in range(index_aux, len(genre_movie)):
                if genres[j] == genre_movie[k]:
                    has_genre.append(1)
                    has = True
                    index_aux += 1
                    break
                    
            if(has == False):
                has_genre.append(0)
        movies_genres.loc[index] = has_genre
        index += 1
                
movies_genres

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
6,7,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
7,8,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,10,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [49]:
user_rating

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [51]:
movies.iloc[:, [0, 1]]

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [140]:
movies_info = pd.DataFrame(columns=['Movie', 'Type', 'Country', 'Director', 'imdbRating', 'Plot', 'Awards', 'Writer', 'Production'])

index = 0
## Put in a csv when take all the movie info
for i in movies.itertuples(index = False):
    l = []
    search_year = re.search(' \((\d+)\)$', i[1]) 
    year = ""
    if(search_year != None):
        year = search_year.group(1)
    movie_name = re.sub(' \(\d+\)$', '', i[1])
    aux = fetch_movie_info(year = year, movie_name = movie_name)
    l.extend([aux['Title'], aux['Type'], aux['Country'], aux['Director'], aux['imdbRating'], aux['Plot'], aux['Awards'], aux['Writer'], aux['Production']])
    print(l)
    movies_info.loc[index] = l
    index += 1
    #break
    
movies_info

['Toy Story', 'movie', 'USA', 'John Lasseter', '8.3', 'A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that he has life (as a toy) good. However, he must worry about Andy\'s family moving, and what Woody does not know is about Andy\'s birthday party. Woody does not realize that Andy\'s mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy\'s new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips.', 'Nominated for 3 Oscars. Another 23 wins & 17 nominations.', 'John Lasseter (original story by), Pete Docter (original story by), Andrew Stanton (original story by), Joe R

StopIteration: timeout

In [142]:
movies_info

,Movie,Type,Country,Director,imdbRating,Plot,Awards,Writer,Production
0,Toy Story,movie,USA,John Lasseter,8.3,A little boy named Andy loves to be in his roo...,Nominated for 3 Oscars. Another 23 wins & 17 n...,"John Lasseter (original story by), Pete Docter...",Buena Vista
1,Jumanji,movie,USA,Joe Johnston,6.9,After being trapped in a jungle board game for...,4 wins & 9 nominations.,"Jonathan Hensleigh (screenplay), Greg Taylor (...",Sony Pictures Home Entertainment
2,Grumpier Old Men,movie,USA,Howard Deutch,6.6,Things don't seem to change much in Wabasha Co...,2 wins & 2 nominations.,"Mark Steven Johnson (characters), Mark Steven ...",Warner Home Video
3,Waiting to Exhale,movie,USA,Forest Whitaker,5.7,This story based on the best selling novel by ...,8 wins & 9 nominations.,"Terry McMillan (novel), Terry McMillan (screen...",Twentieth Century Fox Home Entertainment
4,Father of the Bride Part II,movie,USA,Charles Shyer,5.9,"In this sequel to ""Father of the Bride"", Georg...",Nominated for 1 Golden Globe. Another 1 win & ...,"Albert Hackett (screenplay ""Father's Little Di...",Disney
5,Heat,movie,USA,Michael Mann,8.2,Hunters and their prey--Neil and his professio...,12 nominations.,Michael Mann,Warner Bros.
6,Sabrina,movie,"Germany, USA",Sydney Pollack,6.3,"While she was growing up, Sabrina Fairchild sp...",Nominated for 2 Oscars. Another 2 wins & 4 nom...,"Samuel A. Taylor (play), Billy Wilder (earlier...",Paramount
